# Draft Assistant

A code-based draft assistant (no UI).

In [1]:
# Reinstall package.
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Show available drafting models
# sd.list_models()

In [3]:
# Load a model. 
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [6]:
# Draft assistant
# collection = ["Vivien Reid", "Llanowar Elves", "Refute", "Bigfin Bouncer"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
collection = []
pick_order = dm.get_pick_order(collection)
pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])]
# pick_order = pick_order[pick_order["color_identity"].isin(["G"])]
# pick_order.sort_values(by="synergy", ascending=False)
pick_order

# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(collection, starting_colors="")

,name,rarity,color_identity,p1p1_rating,synergy,rating
174,Mischievous Mystic,uncommon,U,3.72,0.0,3.72
267,Vampire Nighthawk,uncommon,B,3.65,0.0,3.65
244,Sun-Blessed Healer,uncommon,W,3.52,0.0,3.52
101,Fiery Annihilation,uncommon,R,3.45,0.0,3.45
50,Cat Collector,uncommon,W,3.44,0.0,3.44
134,Hero's Downfall,uncommon,B,3.18,0.0,3.18
0,Abrade,uncommon,R,3.14,0.0,3.14
86,Empyrean Eagle,uncommon,Multicolor,3.13,0.0,3.13
94,Faebloom Trick,uncommon,U,3.07,0.0,3.07
233,Spectral Sailor,uncommon,U,3.07,0.0,3.07


# Sealed/Deckbuilding Assistant

In [5]:
# Paste sealed (or draft) pool from Arena export into notebook
# For detailed instructions on exporting a deck, see: https://draftsim.com/mtg-arena-export-deck/
sealed_pool = """2 Healer's Hawk (FDN) 142
1 Mountain (FDN) 279
2 Joust Through (FDN) 19
1 Ajani's Pridemate (FDN) 135
1 Banishing Light (FDN) 138
1 Dazzling Angel (FDN) 9
1 Inspiring Paladin (FDN) 18
1 Armasaur Guide (FDN) 3
1 An Offer You Can't Refuse (FDN) 160
1 Swamp (FDN) 277
1 Witness Protection (FDN) 168
1 Aetherize (FDN) 151
1 Lightshell Duo (FDN) 157
1 Extravagant Replication (FDN) 154
1 Grappling Kraken (FDN) 39
3 Island (FDN) 275
1 Pilfer (FDN) 181
4 Plains (FDN) 273
1 Frenzied Goblin (FDN) 199
1 Kellan, Planar Trailblazer (FDN) 91
1 Rite of the Dragoncaller (FDN) 92
1 Mild-Mannered Librarian (FDN) 228
3 Forest (FDN) 281
1 Genesis Wave (FDN) 221
1 Scoured Barrens (FDN) 266
1 Dismal Backwater (FDN) 261
1 Swiftwater Cliffs (FDN) 268
1 Jungle Hollow (FDN) 263
1 Wardens of the Cycle (FDN) 125
1 Evolving Wilds (FDN) 262
1 Adventuring Gear (FDN) 249

Sideboard
1 Goblin Boarders (FDN) 87
2 Vampire Soulcaller (FDN) 75
1 Gutless Plunderer (FDN) 60
2 Think Twice (FDN) 165
1 Wary Thespian (FDN) 235
1 Serra Angel (FDN) 147
2 Garruk's Uprising (FDN) 220
1 Stromkirk Bloodthief (FDN) 185
1 Spinner of Souls (FDN) 112
2 Axgard Cavalry (FDN) 189
1 Dwynen's Elite (FDN) 218
1 Involuntary Employment (FDN) 203
1 Helpful Hunter (FDN) 16
2 Erudite Wizard (FDN) 37
1 Burnished Hart (FDN) 250
1 Blanchwood Armor (FDN) 213
1 Phyrexian Arena (FDN) 180
1 Inspiring Paladin (FDN) 18
1 Courageous Goblin (FDN) 82
2 Hungry Ghoul (FDN) 62
1 Prideful Parent (FDN) 21
2 Broken Wings (FDN) 214
1 Campus Guide (FDN) 251
1 Tragic Banshee (FDN) 73
1 Tatyova, Benthic Druid (FDN) 247
1 Inspiration from Beyond (FDN) 43
1 Genesis Wave (FDN) 221
1 Treetop Snarespinner (FDN) 114
2 Fanatical Firebrand (FDN) 195
1 Gleaming Barrier (FDN) 252
1 Bite Down (FDN) 212
1 Soul-Shackled Zombie (FDN) 70
1 Uncharted Voyage (FDN) 53
1 Wardens of the Cycle (FDN) 125
1 Slumbering Cerberus (FDN) 94
1 Drakuseth, Maw of Flames (FDN) 193
1 Strix Lookout (FDN) 52
1 Spitfire Lagac (FDN) 208
1 Stab (FDN) 71
1 Apothecary Stomper (FDN) 99
1 Infernal Vessel (FDN) 63
1 Tolarian Terror (FDN) 167
1 Icewind Elemental (FDN) 42
1 Cackling Prowler (FDN) 101
1 Cephalid Inkmage (FDN) 32
1 Youthful Valkyrie (FDN) 149
"""

In [9]:
# Parse into collection list
sealed_cardnames = sd.parse_cardnames(sealed_pool)

# Get deckbuild recommendations - expect to need to tweak a handful of cards. 
dm = sd.DraftModel(set="FDN")
deck_rec = dm.get_deck_recommendation(sealed_cardnames, starting_colors="BR")
# deck_rec

NameError: name 'sealed_pool' is not defined

In [11]:
csv_path = "../data/17lands/draft_data_public.STX.TradDraft.csv.gz"

import pandas as pd

for draft_chunk in pd.read_csv(csv_path, chunksize=1000, compression="gzip"):
    break

draft_chunk.head() # "event_match_wins" always available as a field. 

,draft_data_public.STX.TradDraft.csv,user_n_matches_bucket,draft_id,draft_time,expansion,event_type,event_match_wins,event_match_losses,pack_number,pick_number,...,pool_Waterfall Aerialist,pool_Weather the Storm,pool_Whirlwind Denial,pool_Witherbloom Apprentice,pool_Witherbloom Campus,pool_Witherbloom Command,pool_Witherbloom Pledgemage,pool_Wormhole Serpent,pool_Zephyr Boots,"pool_Zimone, Quandrix Prodigy"
0,0.54,5,34526521ff0a4f27a3c65e81a13256f3,2021-04-26 06:01:38,STX,TradDraft,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.54,5,34526521ff0a4f27a3c65e81a13256f3,2021-04-26 06:01:38,STX,TradDraft,2,1,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0.54,5,34526521ff0a4f27a3c65e81a13256f3,2021-04-26 06:01:38,STX,TradDraft,2,1,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0.54,5,34526521ff0a4f27a3c65e81a13256f3,2021-04-26 06:01:38,STX,TradDraft,2,1,0,4,...,0,0,0,0,0,0,0,0,0,0
4,0.54,5,34526521ff0a4f27a3c65e81a13256f3,2021-04-26 06:01:38,STX,TradDraft,2,1,0,5,...,0,0,0,0,0,0,0,0,0,0
